# **Capitulo 6 – Ensemble Learning and Random Forests**

# Setup

Como hicimos en NoteBooks anteriores, vamos a definir los tamaños de fuente por defecto para que las figuras queden más bonitas:

In [3]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

Y vamos a crear la carpeta `images/ensembles` (si no existe ya), y definir la función `save_fig()` que se utiliza a través de este Notebook para guardar las figuras:

In [4]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "ensembles"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Histogram-Based Gradient Boosting

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing_labels = train_set["median_house_value"]
housing = train_set.drop("median_house_value", axis=1)

Las clases HGB también tienen dos características interesantes: admiten características categóricas y valores perdidos. Esto simplifica bastante el preprocesamiento. Sin embargo, las características categóricas deben representarse como enteros que van de 0 a un número inferior a max_bins. Para ello puede utilizar un OrdinalEncoder. Por ejemplo, a continuación se muestra cómo construir y entrenar un pipeline completo para el conjunto de datos de California housing presentado anteriormente:


In [36]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder 

hgb_reg = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                            remainder="passthrough"),
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)
hgb_reg.fit(housing, housing_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['ocean_proximity'])])),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(categorical_features=[0],
                                               random_state=42))])

In [37]:
# evaluar las estadísticas RMSE para el modelo hgb_reg

from sklearn.model_selection import cross_val_score

hgb_rmses = -cross_val_score(hgb_reg, housing, housing_labels,
                             scoring="neg_root_mean_squared_error", cv=10)
pd.Series(hgb_rmses).describe()

count       10.000000
mean     47613.307194
std       1295.422509
min      44963.213061
25%      47001.233485
50%      48000.963564
75%      48488.093243
max      49176.368465
dtype: float64

Todo el proceso es tan corto como las importaciones. No hay necesidad de un imputer, escalador, o un one-hot encoder, por lo que es realmente conveniente. Tenga en cuenta que categorical_features debe establecerse en los índices de las columnas categóricas (o en una matriz booleana). Sin ningún ajuste de hiperparámetros, este modelo arroja un RMSE de unos 47.600, lo que no está nada mal.

En el ecosistema Python ML existen otras implementaciones optimizadas de gradient boosting: en particular, XGBoost, Cat-Boost y LightGBM. Estas bibliotecas existen desde hace varios años. Todas están especializadas en gradient boosting, sus APIs son muy similares a las de Scikit-Learn, y proporcionan muchas características adicionales, incluyendo aceleración GPU; ¡definitivamente deberías echarles un vistazo! Por otra parte, la librería TensorFlow Random Forests proporciona implementaciones optimizadas de una variedad de algoritmos de random forest, incluyendo random forest simples, extra-trees, GBRT, y varios más.